In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm


In [ ]:
_ = torch.manual_seed(0)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081))])

mnist_trainset = datasets.MNIST(root='./data',train=True,download=True,transform=transform)

train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10,shuffle=True)

mnist_testset = datasets.MNIST(root='./data',train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset,  batch_size=10, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

class OPStupidNet(nn.Module):
  def __init__(self, hidden_1=1000,hidden_2=2000):
    super(OPStupidNet, self).__init__()
    self.linear1 = nn.Linear(28*28,hidden_1)
    self.linear2 = nn.Linear(hidden_1,hidden_2)
    self.linear3 = nn.Linear(hidden_2,10)
    self.relu = nn.ReLU()

  def forward(self,img):
    x = img.view(-1,28*28)
    x = self.linear3(self.relu(self.linear2(self.relu(self.linear1(x)))))
    return x

snet = OPStupidNet().to(device)


In [ ]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
  Loss = nn.CrossEntropyLoss()
  Optimizer = torch.optim.Adam(snet.parameters(),lr=0.001)
  total_iterations = 0

  for epoch in range(epochs):
    net.train()
    loss_sum = 0
    num_iterations = 0
    data_iterator = tqdm(train_loader,desc=f'Epoch {epoch+1}')
    if total_iterations_limit is not None:
      data_iterator.total = total_iterations_limit
    for data in data_iterator:
      num_iterations += 1
      total_iterations += 1
      x, y = data
      x = x.to(device)
      y = y.to(device)
      Optimizer.zero_grad()
      output = net(x.view(-1,28*28))
      loss = Loss(output,y)
      loss_sum += loss.item()
      avg_loss = loss_sum / num_iterations
      data_iterator.set_postfix(loss=avg_loss)
      loss.backward()
      Optimizer.step()
      if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
        return

train(train_loader,snet, epochs=1)




Epoch 1: 100%|██████████| 6000/6000 [00:38<00:00, 155.02it/s, loss=0.234]


In [ ]:
og_weights = {}
for name,param in snet.named_parameters():
  og_weights[name] = param.clone().detach()


In [ ]:
def test():
  correct = 0
  total = 0
  wa_counts = [0 for i in range(10)]

  with torch.no_grad():
    for data in tqdm(test_loader,desc='Testing'):
      x,y = data
      x = x.to(device)
      y = y.to(device)
      output = snet(x.view(-1,28*28))
      for idx,i in enumerate(output):
        if torch.argmax(i) == y[idx]:
          correct += 1
        else:
          wa_counts[y[idx]] += 1
        total += 1
  print(f'Accuracy: {round(correct/total,3)}')
  for i in range(len(wa_counts)):
    print(f'Wrong count for {i}: {wa_counts[i]}')
test()


Testing: 100%|██████████| 1000/1000 [00:03<00:00, 295.11it/s]

Accuracy: 0.961
Wrong count for 0: 11
Wrong count for 1: 20
Wrong count for 2: 54
Wrong count for 3: 68
Wrong count for 4: 24
Wrong count for 5: 18
Wrong count for 6: 40
Wrong count for 7: 55
Wrong count for 8: 9
Wrong count for 9: 87


#MISC Stuff

In [ ]:
total_params_og = 0
for index, layer in enumerate([snet.linear1,snet.linear2,snet.linear3]):
  total_params_og += layer.weight.nelement() + layer.bias.nelement()
  print(f'Layert {index + 1}: W: {layer.weight.shape} + B: {layer.bias.shape}')
print(f'Total params: {total_params_og}')


Layert 1: W: torch.Size([1000, 784]) + B: torch.Size([1000])
Layert 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000])
Layert 3: W: torch.Size([10, 2000]) + B: torch.Size([10])
Total params: 2807010


#LoRA Params

In [ ]:
class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1, device='cpu'):
        super().__init__()
        # Section 4.1 of the paper:
        # We use a random gaussian initialization for A and zero for B so that DeltaW = BA is zero at the beginning of training.
        self.lora_A = nn.Parameter(torch.zeros((rank, features_out), device=device))
        self.lora_B = nn.Parameter(torch.zeros((features_in, rank), device=device))
        nn.init.normal_(self.lora_A, mean=0, std=1)

        # Scale the low-rank update as alpha / rank.
        self.scale = alpha / rank
        self.enabled = True

    def forward(self, og_weights):
        if self.enabled:
            # Compute the low-rank update and add it to the original weights.
            update = torch.matmul(self.lora_B, self.lora_A).view(og_weights.shape) * self.scale
            print("Applying LoRA update")
            return og_weights + update
        else:
            return og_weights


In [ ]:
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterization(layer, device, rank=1, lora_alpha=1):
    # Only add the parameterization to the weight matrix, ignore the Bias

    # From section 4.2 of the paper:
    #   We limit our study to only adapting the attention weights for downstream tasks and freeze the MLP modules (so they are not trained in downstream tasks) both for simplicity and parameter-efficiency.
    #   [...]
    #   We leave the empirical investigation of [...], and biases to a future work.

    features_in, features_out = layer.weight.shape
    return LoRAParametrization(
        features_in, features_out, rank=rank, alpha=lora_alpha, device=device
    )

parametrize.register_parametrization(
    snet.linear1, "weight", linear_layer_parameterization(snet.linear1, device)
)
parametrize.register_parametrization(
    snet.linear2, "weight", linear_layer_parameterization(snet.linear2, device)
)
parametrize.register_parametrization(
    snet.linear3, "weight", linear_layer_parameterization(snet.linear3, device)
)


def enable_disable_lora(enabled=True):
    for layer in [snet.linear1, snet.linear2, snet.linear3]:
        layer.parametrizations["weight"][0].enabled = enabled

Applying LoRA update
Applying LoRA update
Applying LoRA update


In [ ]:
total_parameters_lora = 0
total_parameters_non_lora = 0
for index, layer in enumerate([snet.linear1, snet.linear2, snet.linear3]):
    total_parameters_lora += layer.parametrizations["weight"][0].lora_A.nelement() + layer.parametrizations["weight"][0].lora_B.nelement()
    total_parameters_non_lora += layer.weight.nelement() + layer.bias.nelement()
    print(
        f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape} + Lora_A: {layer.parametrizations["weight"][0].lora_A.shape} + Lora_B: {layer.parametrizations["weight"][0].lora_B.shape}'
    )
# The non-LoRA parameters count must match the original network
assert total_parameters_non_lora == total_params_og
print(f'Total number of parameters (original): {total_parameters_non_lora:,}')
print(f'Total number of parameters (original + LoRA): {total_parameters_lora + total_parameters_non_lora:,}')
print(f'Parameters introduced by LoRA: {total_parameters_lora:,}')
parameters_incremment = (total_parameters_lora / total_parameters_non_lora) * 100
print(f'Parameters incremment: {parameters_incremment:.3f}%')

Applying LoRA update
Applying LoRA update
Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000]) + Lora_A: torch.Size([1, 784]) + Lora_B: torch.Size([1000, 1])
Applying LoRA update
Applying LoRA update
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000]) + Lora_A: torch.Size([1, 1000]) + Lora_B: torch.Size([2000, 1])
Applying LoRA update
Applying LoRA update
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10]) + Lora_A: torch.Size([1, 2000]) + Lora_B: torch.Size([10, 1])
Total number of parameters (original): 2,807,010
Total number of parameters (original + LoRA): 2,813,804
Parameters introduced by LoRA: 6,794
Parameters incremment: 0.242%


In [ ]:
for name, param in snet.named_parameters():
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# Load the MNIST dataset again, by keeping only the digit 1
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Train the network with LoRA only on the digit 1 and only for 100 batches (hoping that it would improve the performance on the digit 1)
train(train_loader, snet, epochs=1, total_iterations_limit=100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s, loss=0.114]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s, loss=0.152]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


...

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


Epoch 1:  86%|████████▌ | 86/100 [00:00<00:00, 118.25it/s, loss=0.101]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


Epoch 1:  99%|█████████▉| 99/100 [00:00<00:00, 114.86it/s, loss=0.0961]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


In [ ]:
assert torch.all(snet.linear1.parametrizations.weight.original == og_weights['linear1.weight'])
assert torch.all(snet.linear2.parametrizations.weight.original == og_weights['linear2.weight'])
assert torch.all(snet.linear3.parametrizations.weight.original == og_weights['linear3.weight'])

enable_disable_lora(enabled=True)
# The new linear1.weight is obtained by the "forward" function of our LoRA parametrization
# The original weights have been moved to net.linear1.parametrizations.weight.original
# More info here: https://pytorch.org/tutorials/intermediate/parametrizations.html#inspecting-a-parametrized-module
assert torch.equal(snet.linear1.weight, snet.linear1.parametrizations.weight.original + (snet.linear1.parametrizations.weight[0].lora_B @ snet.linear1.parametrizations.weight[0].lora_A) * snet.linear1.parametrizations.weight[0].scale)

enable_disable_lora(enabled=False)
# If we disable LoRA, the linear1.weight is the original one
assert torch.equal(snet.linear1.weight, og_weights['linear1.weight'])

Applying LoRA update


In [ ]:
enable_disable_lora(enabled=True)
test()

Testing:   0%|          | 0/1000 [00:00<?, ?it/s]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


Testing:   2%|▏         | 21/1000 [00:00<00:04, 209.07it/s]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA

Testing:   4%|▍         | 42/1000 [00:00<00:04, 206.13it/s]

...

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA

Testing:  98%|█████████▊| 980/1000 [00:04<00:00, 202.60it/s]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update


Testing: 100%|██████████| 1000/1000 [00:05<00:00, 197.56it/s]

Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Applying LoRA update
Accuracy: 0.894
Wrong count for 0: 14
Wrong count for 1: 23
Wrong count for 2: 65
Wrong count for 3: 130
Wrong count for 4: 330
Wrong count for 5: 162
Wrong count for 6: 54
Wrong count for 7: 195
Wrong count for 8: 74
Wrong count for 9: 14


In [ ]:
enable_disable_lora(enabled=False)
test()

Testing: 100%|██████████| 1000/1000 [00:02<00:00, 351.99it/s]

Accuracy: 0.961
Wrong count for 0: 11
Wrong count for 1: 20
Wrong count for 2: 54
Wrong count for 3: 68
Wrong count for 4: 24
Wrong count for 5: 18
Wrong count for 6: 40
Wrong count for 7: 55
Wrong count for 8: 9
Wrong count for 9: 87
